In [4]:
import pandas as pd
from utils.clean import Clean

clean = Clean()


# Extract

df = pd.read_csv('./data/raw/imoveis.csv')

df.columns

Index(['Unnamed: 0', 'id', 'url', 'header', 'address', 'amenities', 'crawler',
       'crawled_at', 'Quarto', 'Banheiro', 'Garagem', 'condominio', 'preço',
       'area_limpo', 'Guarita blindada', 'Vista exterior', 'Arvorismo',
       'Bicicletário', 'Perto de vias de acesso', 'Piso frio',
       'Armário na cozinha', 'Quadra de squash', 'Cabeamento estruturado',
       'Quadra de tênis', 'Portão eletrônico', 'Janela grande',
       'Perto de transporte público', 'Espaço fitness', 'Academia', 'Recepção',
       'Área de lazer', 'Armário embutido', 'Serviços públicos essenciais',
       'Espaço gourmet', 'Sala de almoço', 'Piscina', 'TV a cabo',
       'Móvel planejado', 'Quadra poliesportiva', 'Mais de um andar', 'Spa',
       'Aceita animais', 'Sala de massagem', 'Perto de shopping',
       'Portaria 24h', 'Espaço Pet', 'Copa', 'Pé direito alto', 'Cozinha',
       'Segurança 24h', 'Cozinha grande', 'Espaço verde / Parque',
       'Acesso para deficientes', 'Dependência de empregados',

In [ ]:
# Transform
def cleancode(df):
    df = df.drop_duplicates(keep="first", subset=[coluna for coluna in df.columns if coluna!="crawled_at"]).reset_index(drop=True)
    filtro_de_anuncios = [str(_id).isnumeric() for _id in df["id"]]
    df = df[filtro_de_anuncios].reset_index(drop=True)
    df["rooms"].str.split(" ").str[0].str.replace("--" , "0")
    df["Quarto"] = (df["rooms"].str.split(" ").str[0].str.replace("--","0").astype(int))
    df["Banheiro"] = (df["bathrooms"].str.split(" ").str[0].str.replace("--","0").astype(int))
    df["Garagem"] = (df["garages"].str.split(" ").str[0].str.replace("--","0").astype(int))
    df["condo"] = df["condo"].fillna("MISSING")
    df["condominio"] = [int(w.split("R$ ")[1].replace(".","")) if w!="MISSING" else np.nan for w in df["condo"]]
    df["preço"] = [int(w.split("R$ ")[1].split(" /")[0].split("\n")[0].replace(".","")) for w in df["price"]]
    df["area_limpo"] = df["area"].astype(int)
    df["crawled_at"] = pd.to_datetime(df["crawled_at"], format="%Y-%m-%d %H:%M")
    df = df.drop(columns = ["area", "rooms", "bathrooms", "garages", "price", "condo"])
    df["bairro"] = (df["address"].str.split("- ").str[1].str.split(",").str[0])
    df.loc[df["bairro"].isin(["RJ"]), "bairro"] = (df.loc[df["bairro"].isin(["RJ"]), "address"].str.split(",").str[0])
    df.loc[df["crawler"]!=df["bairro"],"bairro"].unique()
    df["crawler"] = (df["crawler"].str.lower().str.replace(" ","_"))
    df["bairro"] = (df["bairro"].str.lower().str.replace(" ","_").str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))
    df = df.drop(columns="bairro")
    df["amenities"] = df["amenities"].str.split("\n")
    df["amenities"] = df["amenities"].fillna("")
    amenities = []
    for am in df["amenities"]:
        if am != "":
            amenities = amenities + am
    unico_amenities = list(set(amenities))
    df["amenities"] = df["amenities"].str.join(", ")
    for amenity in unico_amenities:
        print(amenity)
        df[amenity]=0
        df.loc[df["amenities"].str.contains(amenity), amenity] = 1
    return df
cleancode(df)

coluna = "Preço"
df = clean.remove_outliers_bairro(df,coluna,0.20)

coluna = "condominio"
df = clean.remove_outliers_bairro(df,coluna,0.20)

# Load
df.to_csv('./data/trusted/imoveis.csv')